In [346]:
import os
import numpy as np
import time
import copy
import sys

import matplotlib
import matplotlib.pyplot as plt


ang_2_bohr = 1.0/0.52917721067
hart_2_ev = 27.21138602

import atomistic_tools.cp2k_stm_utilities as csu

In [355]:
class Input:
    def __init__(self):
        self.npz_file = "/home/kristjan/local_work/stm-test/morbs_dx0.2.npz"
        self.hartree_file = "/home/kristjan/local_work/stm-test/aiida-HART-v_hartree-1_0.cube"
        self.extrap_plane = 3.0
        self.extrap_extent = 5.0
        self.output_dir = "output"
        self.bias_voltages = [-1.0, 1.0]
        self.stm_plane_heights = [3.0, 5.0]
        self.stm_isovalues = [1e-8, 1e-7]
        self.sts_plane_heights = [3.0, 5.0]
        self.sts_isovalues = [1e-8, 1e-7]
        self.sts_isov_bias = [1.0, -1.0]
        self.sts_de = 0.05
        self.sts_fwhm = 0.1
        self.sts_elim = [-1.0, 1.0]
        self.orb_plane_heights = [3.0, 5.0]
        self.n_homo = 5
        self.n_lumo = 5
        self.skip_data_output = False
        self.skip_figs = False
        
args = Input()

In [356]:
npz_file_data = np.load(args.npz_file)

x_arr = npz_file_data['x_arr']
y_arr = npz_file_data['y_arr']
z_arr = npz_file_data['z_arr']
mol_bbox = npz_file_data['mol_bbox']
elim = npz_file_data['elim']
ref_energy = npz_file_data['ref_energy']
geom_label = npz_file_data['geom_label']

morb_grids = [npz_file_data['morb_grids_s1']]
morb_energies = [npz_file_data['morb_energies_s1']]
homo_inds = [npz_file_data['homo_s1']]

if 'morb_grids_s2' in npz_file_data:
    morb_grids.append(npz_file_data['morb_grids_s2'])
    morb_energies.append(npz_file_data['morb_energies_s2'])
    homo_inds.append(npz_file_data['homo_s2'])

eval_reg_size = np.array([x_arr[-1] - x_arr[0], y_arr[-1] - y_arr[0], z_arr[-1] - z_arr[0]])
eval_reg_size_n = morb_grids[0][0].shape

if eval_reg_size_n[2] == 1:
    print("Note: only a single plane was evaluated, const-current output will be skipped.")
    dv = np.array([x_arr[1] - x_arr[0], y_arr[1] - y_arr[0], x_arr[1] - x_arr[0]])
else:
    dv = np.array([x_arr[1] - x_arr[0], y_arr[1] - y_arr[0], z_arr[1] - z_arr[0]])

nspin = len(morb_grids)

# z_arr with respect to topmost atom
z_arr -= mol_bbox[5]

x_grid, y_grid = np.meshgrid(x_arr, y_arr, indexing='ij')
x_grid /= ang_2_bohr
y_grid /= ang_2_bohr

def get_plane_index(z, z_arr):
    if z < z_arr[0] or z > z_arr[-1]:
        print("Warning: z outside of z_arr!")
    return (np.abs(z_arr - z)).argmin()

# Size of all the figures:
figure_size = 4
figure_size_xy = (figure_size*eval_reg_size[0]/eval_reg_size[1]+1.0, figure_size)

In [357]:
### -----------------------------------------
### EXTRAPOLATION
### -----------------------------------------

time1 = time.time()
hart_cube_data = csu.read_cube_file(args.hartree_file)
print("Read hartree: %.3f" % (time.time()-time1))

# Get Hartree plane, convert to eV and shift by ref (so it matches morb_energies) 
hart_plane = csu.get_hartree_plane_above_top_atom(hart_cube_data, args.extrap_plane)*hart_2_ev - ref_energy

print("Hartree on extrapolation plane: min: %.4f; max: %.4f; avg: %.4f (eV)" % (
                                                np.min(hart_plane),
                                                np.max(hart_plane),
                                                np.mean(hart_plane)))

if not args.skip_figs:
    plt.figure(figsize=figure_size_xy)
    plt.pcolormesh(hart_plane.T, cmap='seismic')
    plt.colorbar()
    plt.axis('scaled')
    plt.savefig(args.output_dir+"/hartree.png", dpi=300, bbox_inches='tight')
    plt.close()


extrap_plane_index = get_plane_index(args.extrap_plane*ang_2_bohr, z_arr)
if extrap_plane_index >= eval_reg_size_n[2]:
    print("Error: the extrapolation plane can't be outside the initial box (z_max = %.2f)"
           % (z_arr[-1]/ang_2_bohr))
    exit(1)

total_morb_grids = []
for ispin in range(nspin):
    # ready the correct total grid to reduce memory usage...
    tot_g_shape = np.array(morb_grids[ispin][:, :, :, :extrap_plane_index+1].shape)
    tot_g_shape[3] += int(args.extrap_extent*ang_2_bohr/dv[2])
    total_morb_grid = np.zeros(tot_g_shape)

    total_morb_grid[:, :, :, :extrap_plane_index+1] = morb_grids[ispin][:, :, :, :extrap_plane_index+1]
    # free old grid
    morb_grids[ispin] = None

    # Could probably use larger dz for extrapolation to reduce memory usage...
    csu.extrapolate_morbs(total_morb_grid[:, :, :, extrap_plane_index],
                            morb_energies[ispin], dv,
                            args.extrap_extent*ang_2_bohr, False,
                            hart_plane=hart_plane/hart_2_ev,
                            use_weighted_avg=True,
                            output_array=total_morb_grid[:, :, :, extrap_plane_index+1:])
    
    total_morb_grids.append(total_morb_grid)

extended_region_n = np.shape(total_morb_grids[0])

# In bohr and wrt topmost atom
total_z_arr = np.arange(0.0, extended_region_n[3]*dv[2], dv[2]) + z_arr[0]

Read hartree: 0.601
Hartree on extrapolation plane: min: 4.5211; max: 4.8803; avg: 4.7747 (eV)
Extrapolation time: 5.048 s


In [358]:
### -----------------------------------------
### Plotting methods
### -----------------------------------------

class FormatScalarFormatter(matplotlib.ticker.ScalarFormatter):
    def __init__(self, fformat="%1.1f", offset=True, mathText=True):
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,
                                                        useMathText=mathText)
    def _set_format(self, vmin, vmax):
        self.format = self.fformat
        if self._useMathText:
            self.format = '$%s$' % matplotlib.ticker._mathdefault(self.format)


def make_plot(data, fpath, title=None, center0=False, vmin=None, vmax=None, cmap='gist_heat'):
    if not isinstance(data, (list,)):
        data = [data]
    if not isinstance(center0, (list,)):
        center0 = [center0]
    if not isinstance(title, (list,)):
        title = [title]

    plt.figure(figsize=(figure_size_xy[0], len(data)*figure_size_xy[1]))
    for i, data_e in enumerate(data):
        plt.subplot(len(data), 1, i+1)
        if center0[i]:
            data_amax = np.max(np.abs(data_e))
            plt.pcolormesh(x_grid, y_grid, data_e, vmin=-data_amax, vmax=data_amax, cmap=cmap)
        else:
            plt.pcolormesh(x_grid, y_grid, data_e, vmin=vmin, vmax=vmax, cmap=cmap)
        plt.xlabel("x (angstrom)")
        plt.ylabel("y (angstrom)")
        if 1e-3 < np.max(data) < 1e3:
            cb = plt.colorbar()
        else:
            cb = plt.colorbar(format=FormatScalarFormatter("%.1f"))
        cb.formatter.set_powerlimits((-2, 2))
        cb.update_ticks()
        if i < len(title):
            plt.title(title[i], loc='left')
    plt.axis('scaled')
    plt.savefig(fpath, dpi=300, bbox_inches='tight')
    plt.close()

def make_series_plot(data, fpath):
    plt.figure(figsize=(figure_size_xy[0]*len(args.bias_voltages), figure_size_xy[1]))
    for i_bias, bias in enumerate(args.bias_voltages):
        plt.subplot(1, len(args.bias_voltages), i_bias+1)
        plt.pcolormesh(x_grid, y_grid, data[:, :, i_bias], cmap='gist_heat')
        plt.axis('scaled')
        plt.title("V=%.2f"%bias)
        plt.xticks([])
        plt.yticks([])
    plt.savefig(fpath, dpi=300, bbox_inches='tight')
    plt.close()

In [359]:
### -----------------------------------------
### Saving HOMO and LUMO
### -----------------------------------------

orbital_data = np.ones((
    nspin,
    len(args.orb_plane_heights),
    args.n_homo + args.n_lumo,
    x_grid.shape[0], x_grid.shape[1])) * (-1.0)

for i_height, plane_height in enumerate(args.orb_plane_heights):
    plane_index = get_plane_index(plane_height*ang_2_bohr, total_z_arr)
    if plane_index < 0:
        print("Height %.1f is outside evaluation range, skipping." % plane_height)
        continue
    for ispin in range(nspin):
        for i_h in range(-args.n_homo+1, args.n_lumo+1):
            ind = homo_inds[ispin] + i_h
            if ind > len(morb_energies[ispin]) - 1:
                print("Homo %d is out of energy range, ignoring" % i_h)
                continue
            orb_data = total_morb_grids[ispin][ind][:, :, plane_index]
            orbital_data[ispin, i_height, i_h+args.n_homo-1] = orb_data
            
            if not args.skip_figs:
                fpath = args.output_dir+"/orb_h%.1f_s%d_%02dhomo%d.png"%(plane_height, ispin, i_h+args.n_homo-1, i_h)
                title = "homo %d, E=%.6f" % (i_h, morb_energies[ispin][ind])
                make_plot([orb_data, orb_data**2], fpath, title=[title, "square"], center0=[True, False], cmap='seismic')

if not args.skip_data_output:
    fpath = args.output_dir+"/orbs.npz"
    np.savez(fpath, data=orbital_data, heights=args.orb_plane_heights, ihomo=args.n_homo-1,
        x=x_arr/ang_2_bohr, y=y_arr/ang_2_bohr)

In [360]:
### -----------------------------------------
### Summing charge densities according to bias voltages
### -----------------------------------------

# Sum up both spins

charge_dens_arr = np.zeros((len(args.bias_voltages),
                            extended_region_n[1],
                            extended_region_n[2],
                            extended_region_n[3]))

# NB Homo is only added to negative bias !
for i_bias, bias in enumerate(args.bias_voltages):
    for ispin in range(nspin):
        for imo, morb_grid in enumerate(total_morb_grids[ispin]):
            if morb_energies[ispin][imo] > np.max([0.0, bias]):
                break
            if morb_energies[ispin][imo] > np.min([0.0, bias]):
                charge_dens_arr[i_bias, :, :, :] += morb_grid**2


In [361]:
### -----------------------------------------
### Constant height STM
### -----------------------------------------

time1 = time.time()

const_height_data = np.zeros((
    len(args.stm_plane_heights),
    x_grid.shape[0], x_grid.shape[1],
    len(args.bias_voltages)))

for i_height, plane_height in enumerate(args.stm_plane_heights):
    for i_bias, bias in enumerate(args.bias_voltages):
        plane_index = get_plane_index(plane_height*ang_2_bohr, total_z_arr)
        const_height_data[i_height, :, :, i_bias] = charge_dens_arr[i_bias][:, :, plane_index]

    if not args.skip_figs:
        for i_bias, bias in enumerate(args.bias_voltages):
            fpath = args.output_dir+"/stm_ch_v%.2f_h%.1f.png"%(bias, plane_height)
            title = "h = %.1f; V = %.2f" % (plane_height, bias)
            make_plot(const_height_data[i_height, :, :, i_bias], fpath, title=title)
        
        fpath = args.output_dir+"/stm_ch_h%.1f.png"%(plane_height)
        make_series_plot(const_height_data[i_height], fpath)

if not args.skip_data_output:
    fpath = args.output_dir+"/stm_ch.npz"
    np.savez(fpath, data=const_height_data, heights=args.stm_plane_heights, bias=args.bias_voltages,
        x=x_arr/ang_2_bohr, y=y_arr/ang_2_bohr)

print("Time taken to create CH images: %.1f" % (time.time() - time1))

Time taken to create CH images: 2.8


In [362]:
### -----------------------------------------
### Constant current STM
### -----------------------------------------

def get_isosurf(data, value, z_vals, interp=True):
    rev_data = data[:, :, ::-1]
    rev_z_vals = z_vals[::-1]

    # Add a zero-layer at start to make sure we surpass it
    zero_layer = np.zeros((data.shape[0], data.shape[1], 1))
    rev_data = np.concatenate((zero_layer, rev_data), axis=2)
    rev_z_vals = np.concatenate(([rev_z_vals[0]], rev_z_vals))

    # Find first index that surpasses the isovalue
    indexes = np.argmax(rev_data > value, axis=2)
    # If an index is 0, no values in array are bigger than the specified
    num_surpasses = (indexes == 0).sum()
    if num_surpasses != 0:
        print("Warning: The isovalue %.3e was not reached for %d pixels" % (value, num_surpasses))
    # Set surpasses as the bottom surface
    indexes[indexes == 0] = len(z_vals) - 1

    if interp:
        z_val_plane = np.ones(np.shape(rev_data)[0:2])*z_vals[0]
        for ix in range(np.shape(rev_data)[0]):
            for iy in range(np.shape(rev_data)[1]):
                ind = indexes[ix, iy]
                if ind == len(z_vals) - 1:
                    continue
                val_g = rev_data[ix, iy, ind]
                z_val_g = rev_z_vals[ind]
                val_s = rev_data[ix, iy, ind - 1]
                z_val_s = rev_z_vals[ind - 1]
                z_val_plane[ix, iy] = (value - val_s)/(val_g - val_s)*(z_val_g - z_val_s) + z_val_s
        return z_val_plane
    return rev_z_vals[indexes]

def get_isosurf_indexes(data, value, interp=True):
    rev_data = data[:, :, ::-1]
    
    # Add a zero-layer at start to make sure we surpass it
    zero_layer = np.zeros((data.shape[0], data.shape[1], 1))
    rev_data = np.concatenate((zero_layer, rev_data), axis=2)
    
    nz = rev_data.shape[2]

    # Find first index that surpasses the isovalue
    indexes = np.argmax(rev_data > value, axis=2)
    # If an index is 0, no values in array are bigger than the specified
    num_surpasses = (indexes == 0).sum()
    if num_surpasses != 0:
        print("Warning: The isovalue %.3e was not reached for %d pixels" % (value, num_surpasses))
    # Set surpasses as the bottom surface
    indexes[indexes == 0] = nz - 1
    
    if interp:
        indexes_float = indexes.astype(float)
        for ix in range(np.shape(rev_data)[0]):
            for iy in range(np.shape(rev_data)[1]):
                ind = indexes[ix, iy]
                if ind == nz - 1:
                    continue
                val_g = rev_data[ix, iy, ind]
                val_s = rev_data[ix, iy, ind - 1]
                indexes_float[ix, iy] = ind - (val_g-value)/(val_g-val_s)
        return nz - indexes_float - 1
    return nz - indexes.astype(float) - 1

def index_with_interpolation(index_arr, array):
    i = index_arr.astype(int)
    remain = index_arr-i
    iplus = np.clip(i+1, a_min=None, a_max=len(array)-1)
    return array[iplus]*remain +(1-remain)*array[i]

if eval_reg_size_n[2] != 1:

    time1 = time.time()

    const_curr_data = np.zeros((
        len(args.stm_isovalues),
        x_grid.shape[0], x_grid.shape[1],
        len(args.bias_voltages)))

    for i_iso, isovalue in enumerate(args.stm_isovalues):
        for i_bias, bias in enumerate(args.bias_voltages):
            const_cur_imag = index_with_interpolation(
                    get_isosurf_indexes(charge_dens_arr[i_bias], isovalue, True),
                    total_z_arr)
            const_curr_data[i_iso, :, :, i_bias] = const_cur_imag/ang_2_bohr

        if not args.skip_figs:
            for i_bias, bias in enumerate(args.bias_voltages):
                fpath = args.output_dir+"/stm_cc_v%.2f_i%.1e.png"%(bias, isovalue)
                title = "isov = %.1e; V = %.2f" % (isovalue, bias)
                make_plot(const_curr_data[i_iso, :, :, i_bias], fpath, title=title)
            
            fpath = args.output_dir+"/stm_cc_i%.1e.png"%(isovalue)
            make_series_plot(const_curr_data[i_iso], fpath)

    if not args.skip_data_output:
        fpath = args.output_dir+"/stm_cc.npz"
        np.savez(fpath, data=const_curr_data, isovals=args.stm_isovalues, bias=args.bias_voltages,
            x=x_arr/ang_2_bohr, y=y_arr/ang_2_bohr)

    print("Time taken to create CC images: %.1f" % (time.time() - time1))


Time taken to create CC images: 2.4


In [335]:
### -----------------------------------------
### STS
### -----------------------------------------

time1 = time.time()

if len(args.sts_elim) != 2:
    e_arr = np.arange(elim[0], elim[1]+args.sts_de, args.sts_de)
else:
    e_arr = np.arange(args.sts_elim[0], args.sts_elim[1]+args.sts_de, args.sts_de)

def calculate_ldos(de, fwhm, index, e_arr, broad_type='g'):
    """
    Calculates the 2D local density of states map
    index either defines a constant-height plane (single int)
    or a non-flat topography (2d np.array of indexes)
    """
    def lorentzian(x):
        gamma = 0.5*fwhm
        return gamma/(np.pi*(x**2+gamma**2))
    def gaussian(x):
        sigma = fwhm/2.3548
        return np.exp(-x**2/(2*sigma**2))/(sigma*np.sqrt(2*np.pi))
    pldos = np.zeros((eval_reg_size_n[0], eval_reg_size_n[1], len(e_arr)))
    for ispin in range(nspin):
        for i_mo, morb_grid in enumerate(total_morb_grids[ispin]):
            en = morb_energies[ispin][i_mo]
            if isinstance(index, np.ndarray):
                morb_plane = index_with_interpolation_3d(index, morb_grid**2)
            else:
                morb_plane = morb_grid[:, :, plane_index]**2
            if broad_type == 'l':
                morb_ldos_broad = np.einsum('ij,k', morb_plane, lorentzian(e_arr - en))
            else:
                morb_ldos_broad = np.einsum('ij,k', morb_plane, gaussian(e_arr - en))
            pldos += morb_ldos_broad
    return pldos

### -----------------------------------------
### STS CH
### -----------------------------------------

sts_ch_data = np.zeros((
    len(args.sts_plane_heights),
    x_grid.shape[0], x_grid.shape[1],
    len(e_arr)))

for i_height, plane_height in enumerate(args.sts_plane_heights):
    plane_index = get_plane_index(plane_height*ang_2_bohr, total_z_arr)

    pldos = calculate_ldos(args.sts_de, args.sts_fwhm, plane_index, e_arr)
    sts_ch_data[i_height, :, :, :] = pldos

    if not args.skip_figs:
        max_val = np.max(pldos)
        min_val = np.min(pldos)

        for i, energy in enumerate(e_arr):
            fpath = args.output_dir+"/sts_h%.2f_nr%d.png"%(plane_height, i)
            title = "h = %.2f; U = %.3f" % (plane_height, energy)
            make_plot(pldos[:, :, i], fpath, title=title, vmin=min_val, vmax=max_val,  cmap='seismic')
            
if not args.skip_data_output:
    fpath = args.output_dir+"/sts_ch.npz"
    np.savez(fpath, data=sts_ch_data, heights=args.sts_plane_heights, e=e_arr,
        x=x_arr/ang_2_bohr, y=y_arr/ang_2_bohr)

print("Time taken to create CH-STS images: %.1f" % (time.time() - time1))


Time taken to create STS images: 37.3


In [402]:
### -----------------------------------------
### STS CC
### -----------------------------------------

def take_2d_from_3d(val_arr,z_indices):
    # Get number of columns and rows in values array
    nx, ny, nz = val_arr.shape
    # Get linear indices 
    idx = z_indices + nz*np.arange(ny) + nz*ny*np.arange(nx)[:,None]
    return val_arr.flatten()[idx]

def index_with_interpolation_3d(index_arr, array_3d):
    i = index_arr.astype(int)
    remain = index_arr-i
    iplus = np.clip(i+1, a_min=None, a_max=array_3d.shape[2]-1)
    return take_2d_from_3d(array_3d, iplus)*remain +(1-remain)*take_2d_from_3d(array_3d, i)

time1 = time.time()

if len(args.sts_isovalues) != len(args.sts_isov_bias):
    print("STS isovalues and bias voltages don't match, skipping CC-STS.")
else:
    sts_cc_data = np.zeros((
        len(args.sts_isovalues),
        x_grid.shape[0], x_grid.shape[1],
        len(e_arr)))
    
    for i_iso, isoval in enumerate(args.sts_isovalues):
        
        bias = args.sts_isov_bias[i_iso]
        try:
            i_bias = args.bias_voltages.index(bias)
        except:
            print("Skipping CC-STS for bias %.2f." % bias)
            continue
        
        i_isosurf = get_isosurf_indexes(charge_dens_arr[i_bias], isoval, True)

        pldos = calculate_ldos(args.sts_de, args.sts_fwhm*2, i_isosurf, e_arr)
        sts_cc_data[i_iso, :, :, :] = pldos

        if not args.skip_figs:
            max_val = np.max(pldos)
            min_val = np.min(pldos)

            for i, energy in enumerate(e_arr):
                fpath = args.output_dir+"/sts_iso%.1e_v%.1f_nr%d.png"%(isoval, bias, i)
                title = "iso = %.1e; v = %.1fV;  U = %.2f" % (isoval, bias, energy)
                make_plot(pldos[:, :, i], fpath, title=title, vmin=min_val, vmax=max_val,  cmap='seismic')

    if not args.skip_data_output:
        fpath = args.output_dir+"/sts_cc.npz"
        np.savez(fpath, data=sts_cc_data, isov=args.sts_isovalues, bias=args.sts_isov_bias, e=e_arr,
            x=x_arr/ang_2_bohr, y=y_arr/ang_2_bohr)
    
print("Time taken to create CC-STS images: %.1f" % (time.time() - time1))

Time taken to create CC-STS images: 45.8
